In [3]:
import pandas as pd

## Read and merge data

In [27]:
train_hate = pd.read_csv('./datas/train.hate.csv')
train_hate.rename(columns={'label': 'hate_label'}, inplace=True)

In [28]:
train_newstitle = pd.read_csv('./datas/train.news_title.txt', sep='\t', names=['news_title'])
train_newstitle.tail()

,news_title
7891,"허지웅, 허투루 넘길 말 없었다…솔직하게 드러냈던 속사정"
7892,"이혜경, ‘오! 캐롤’ 공연 중 남편 오정욱 부고…오열 속 발인 [종합]"
7893,"'설경구♥' 송윤아, 아들과 즐거운 하루 ""전 엄마니까요"""
7894,"[SC현장]""연예인 인생 협박 유감""…미소잃은 최민수, '보복운전 혐의' 2차 공판..."
7895,"[단독]스태프 사망사고 '서른이지만', 결국 오늘 촬영 취소"


In [29]:
train_gender = pd.read_csv('./datas/train.gender_bias.binary.csv')
train_gender.rename(columns={'label': 'gender_label'}, inplace=True)

In [30]:
train_bias = pd.read_csv('./datas/train.bias.ternary.csv')
train_bias.rename(columns={'label': 'bias_label'}, inplace=True)

In [31]:
from functools import reduce

df = [train_bias, train_gender, train_hate]

train = reduce(lambda left, right: pd.merge(left, right, on=['comments']), df)
train.tail()

,comments,bias_label,gender_label,hate_label
7891,힘내세요~ 응원합니다!!,none,False,none
7892,힘내세요~~삼가 고인의 명복을 빕니다..,none,False,none
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,none,False,none
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,none,False,none
7895,힘들면 관뒀어야지 그게 현명한거다,none,False,none


In [32]:
train = pd.merge(train, train_newstitle, left_index=True, right_index=True)
train.tail()

,comments,bias_label,gender_label,hate_label,news_title
7891,힘내세요~ 응원합니다!!,none,False,none,"허지웅, 허투루 넘길 말 없었다…솔직하게 드러냈던 속사정"
7892,힘내세요~~삼가 고인의 명복을 빕니다..,none,False,none,"이혜경, ‘오! 캐롤’ 공연 중 남편 오정욱 부고…오열 속 발인 [종합]"
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,none,False,none,"'설경구♥' 송윤아, 아들과 즐거운 하루 ""전 엄마니까요"""
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,none,False,none,"[SC현장]""연예인 인생 협박 유감""…미소잃은 최민수, '보복운전 혐의' 2차 공판..."
7895,힘들면 관뒀어야지 그게 현명한거다,none,False,none,"[단독]스태프 사망사고 '서른이지만', 결국 오늘 촬영 취소"


In [33]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7896 entries, 0 to 7895
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   comments      7896 non-null   object
 1   bias_label    7896 non-null   object
 2   gender_label  7896 non-null   bool  
 3   hate_label    7896 non-null   object
 4   news_title    7893 non-null   object
dtypes: bool(1), object(4)
memory usage: 636.1+ KB


In [34]:
is_NaN = train.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = train[row_has_NaN]
rows_with_NaN

,comments,bias_label,gender_label,hate_label,news_title
1522,남의 일에는 신경안써' 하고 쿨한척 하던 인간들. 강남한테는 악플 쓰고 있음ㅋㅋㅋ ...,none,False,hate,NaN
5574,"응애 응애 엄마 저 맘에 안들죠? ........아들 ?? "" 너 내가 우스워 보이...",none,False,hate,NaN
7150,초콜릿' 드럽게 재미없다 ㅠㅠ 다들 공감하실듯,none,False,offensive,NaN


In [35]:
train['news_title'].fillna("No Title", inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7896 entries, 0 to 7895
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   comments      7896 non-null   object
 1   bias_label    7896 non-null   object
 2   gender_label  7896 non-null   bool  
 3   hate_label    7896 non-null   object
 4   news_title    7896 non-null   object
dtypes: bool(1), object(4)
memory usage: 636.1+ KB


## Data Cleansing (comments and news_title column)
- remove punctuation marks + all spaces
- correct spacing between words
- classify part of speech

In [36]:
import string

## remove basic punctuation
def remove_punc(text):
    text_nopunc = "".join([char for char in text if char not in string.punctuation])
    return text_nopunc

train['comments'] = train['comments'].apply(lambda x: remove_punc(x))
train['news_title'] = train['news_title'].apply(lambda x: remove_punc(x))

In [37]:
import re

# remove all punctuations except korean, english, and number
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[a-zA-Z]', '', text)
    text = re.sub('[ㄱ-ㅎㅏ-ㅣ]+', '', text)
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)
    return text

train['comments'] = train['comments'].apply(cleanse)
train['news_title'] = train['news_title'].apply(cleanse)
train.head()

,comments,bias_label,gender_label,hate_label,news_title
0,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,others,False,hate,밤새 조문 행렬故 전미선 동료들이 그리워하는 따뜻한 배우 종합
1,한국적인 미인의 대표적인 분너무나 곱고아름다운모습그모습뒤의 슬픔을 미처 알지못했네요,none,False,none,연중 故 전미선 생전 마지막 미공개 인터뷰환하게 웃는 모습 먹먹종합
2,못된 넘들남의 고통을 즐겼던 넘들이젠 마땅한 처벌을 받아야지그래야 공정한 사회지심은...,none,False,hate,단독 잔나비 라디오 출연 취소→한밤 방송 연기비판 여론 종합
3,12화 어설펐는데 34화 지나서부터는 갈수록 너무 재밌던데,none,False,none,아스달 연대기 장동건김옥빈 들끓는 욕망커플→눈물범벅 칼끝 대립
4,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,gender,True,hate,이슈 구하라 비보’ 최종범 항소심에 영향법조계 “공소권 없음’ 아냐”


In [38]:
from konlpy.tag import Okt
t = Okt()

# part of speech
comment_pos = []
for comment in train['comments']:
    comment_pos.append(t.pos(comment))
train['comment_pos'] = comment_pos

title_pos = []
for tit in train['news_title']:
    title_pos.append(t.pos(tit))
train['title_pos'] = title_pos
train.head()

,comments,bias_label,gender_label,hate_label,news_title,comment_pos,title_pos
0,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,others,False,hate,밤새 조문 행렬故 전미선 동료들이 그리워하는 따뜻한 배우 종합,"[(현재, Noun), (호텔, Noun), (주인, Noun), (심정, Noun...","[(밤새, Noun), (조문, Noun), (행렬, Noun), (故, Forei..."
1,한국적인 미인의 대표적인 분너무나 곱고아름다운모습그모습뒤의 슬픔을 미처 알지못했네요,none,False,none,연중 故 전미선 생전 마지막 미공개 인터뷰환하게 웃는 모습 먹먹종합,"[(한국, Noun), (적, Suffix), (인, Josa), (미인, Noun...","[(연중, Noun), (故, Foreign), (전미선, Noun), (생전, N..."
2,못된 넘들남의 고통을 즐겼던 넘들이젠 마땅한 처벌을 받아야지그래야 공정한 사회지심은...,none,False,hate,단독 잔나비 라디오 출연 취소→한밤 방송 연기비판 여론 종합,"[(못, VerbPrefix), (된, Verb), (넘, Verb), (들, Ve...","[(단독, Noun), (잔나비, Noun), (라디오, Noun), (출연, No..."
3,12화 어설펐는데 34화 지나서부터는 갈수록 너무 재밌던데,none,False,none,아스달 연대기 장동건김옥빈 들끓는 욕망커플→눈물범벅 칼끝 대립,"[(12, Number), (화, Noun), (어설펐는데, Adjective), ...","[(아스, Noun), (달, Noun), (연대기, Noun), (장동건, Nou..."
4,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,gender,True,hate,이슈 구하라 비보’ 최종범 항소심에 영향법조계 “공소권 없음’ 아냐”,"[(1, Number), (사람, Noun), (얼굴, Noun), (손톱, Nou...","[(이슈, Noun), (구하라, Noun), (비보, Noun), (’, Punc..."


In [39]:
train.to_csv('./datas/train_ver2')